# `virtual`: Parquet Demo

In [ ]:
%reload_ext autoreload
%autoreload 2

import virtual

parquet_filepath = 'example.parquet'

### One-shot table compression

In [ ]:
# Save in Parquet format.
virtual.to_format(parquet_filepath, 'example_virtual.parquet', model_types=['sparse-lr'], prefix='demo-debug/')

### Compare to vanilla `Parquet`

In [ ]:
import os

print(f'Parquet: {os.path.getsize('example.parquet')} bytes')
print(f'Virtual: {os.path.getsize('example_virtual.parquet')} bytes')

### Exploring functions

In [ ]:
# Drill the functions.
functions = virtual.train(parquet_filepath, model_types=['sparse-lr'], prefix='./demo-debug/')

functions

### Compress with functions

In [ ]:
# Use them for compression.
virtual.to_format(parquet_filepath, 'example_virtual.parquet', functions)

### Read the file

In [ ]:
df = virtual.from_format('example_virtual.parquet')

### Query the vanilla Parquet file

In [ ]:
import duckdb

duckdb.sql('select avg("Total Earnings") from read_parquet("example.parquet")').df()

### Query the virtualized Parquet file

In [ ]:
virtual.query(
  'select avg("Total Earnings") from read_parquet("example_virtual.parquet")',
  engine = 'duckdb'
)